<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
! pip -q install transformers
! pip -q install parsivar

     |████████████████████████████████| 1.1MB 3.4MB/s 
     |████████████████████████████████| 3.0MB 18.0MB/s 
     |████████████████████████████████| 1.1MB 41.9MB/s 
     |████████████████████████████████| 890kB 53.8MB/s 
     |████████████████████████████████| 36.2MB 83kB/s 
     |████████████████████████████████| 1.5MB 55.2MB/s 


In [2]:
import torch
from __future__ import unicode_literals
# from hazm import *
from parsivar import *
from pprint import pprint
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from scipy import spatial
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForTokenClassification
parsivar_tokenizer = Tokenizer()

In [3]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
# tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model = AutoModelForTokenClassification.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

# **Tokenize**

### Example

In [4]:
tweet = """ بابا بیخیال :)) همون سال اتلتیکو با گل افساید به بایرن رفت فینال.

مشکل داوری نداشتیم اون بازی که میگی.
فصل قبلش هم ۴ ۰ با هاینکس زدیم شما رو
"""

In [5]:
input_sentence = tokenizer.tokenize(tweet)
print(input_sentence)

['بابا', 'بیخیال', ':', ')', ')', 'همون', 'سال', 'اتلتیکو', 'با', 'گل', 'افساید', 'به', 'بایرن', 'رفت', 'فینال', '.', 'مشکل', 'داوری', 'نداشتیم', 'اون', 'بازی', 'که', 'میگی', '.', 'فصل', 'قبلش', 'هم', '۴', '۰', 'با', 'هاینکس', 'زدیم', 'شما', 'رو']


### Tokenize All tweets

In [6]:
! wget "https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/extracted_data.txt"

--2020-09-30 10:13:56--  https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/extracted_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15582302 (15M) [text/plain]
Saving to: ‘extracted_data.txt’

extracted_data.txt  100%[===================>]  14.86M  38.9MB/s    in 0.4s    

2020-09-30 10:13:57 (38.9 MB/s) - ‘extracted_data.txt’ saved [15582302/15582302]



In [7]:
! head extracted_data.txt

سلام بر آن بانوی بزرگوار س
صبح_بخیر 

**************

خدایا به ما توفیق فروکردن کلید آنخ ماعو از فیلم یوزارسیف رو به تحال این شیخ کذاب بده.

**************

خدایا به ما توفیق درآوردن ش ل و ا ر  از پ ا ی این شیخ کذاب، بده.


In [8]:
tweets_str = ""
with open("extracted_data.txt") as file:
  for line in file.readlines():
    tweets_str += line
tweets_str[:20]

'سلام بر آن بانوی بزر'

In [13]:
tweets = tweets_str.split("\n**************\n")
tweets[0], len(tweets)

('سلام بر آن بانوی بزرگوار س\nصبح_بخیر \n', 54866)

In [10]:
tweets_tokens = []
for tweet in tweets:
  tokens = tokenizer.tokenize(tweet)
  print(tweet)
  break

سلام بر آن بانوی بزرگوار س
صبح_بخیر 



# **Model**

In [11]:
from transformers import pipeline

In [12]:
pars_ner = pipeline('ner', model=model, tokenizer=tokenizer)
pars_ner(tweet)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[]